# Segmenting and Clustering Neighbourhoods in Toronto

Importing libraries 

In [20]:
import pandas as pd
import numpy as np
import requests

Get the HTML page of Wiki, and using read_html we convert the html data into list of Data frame objects.

Remove cells which have borrow not assigned.


In [21]:
wiki = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki)
wiki_raw = pd.read_html(wiki_page.content)[0]
df=wiki_raw.melt()#Melting grid to single column
df=df[df.value.str.contains("Not assigned")==False]
df.reset_index(inplace = True)
df.drop(columns = ['index', 'variable'])
df['Postal Code'] = df['value'].map(lambda x: x[0:3])
df['value']=df['value'].map(lambda x: x[3:])
df[['Borough','Neighborhood','c']] =df.value.str.split('(',expand=True)
df['Neighborhood']=df['Neighborhood'].str.replace(')', '', regex=False)
df['Neighborhood']=df['Neighborhood'].str.replace('/', ',', regex=False)
df.drop(columns = ['variable','value','c'],inplace = True)

In [22]:
df.groupby(['Postal Code']).first()

,index,Borough,Neighborhood
Postal Code,,,
M1B,1,Scarborough,"Malvern , Rouge"
M1C,2,Scarborough,"Rouge Hill , Port Union , Highland Creek"
M1E,3,Scarborough,"Guildwood , Morningside , West Hill"
M1G,4,Scarborough,Woburn
M1H,5,Scarborough,Cedarbrae
...,...,...,...
M9N,170,York,Weston
M9P,171,Etobicoke,Westmount
M9R,172,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."


The new Wiki link already has nieghborhood merged according to Postal Code and Borough


In [23]:
len(df['Postal Code'].unique())

103

In [24]:
df[df['Borough'] == 'Not assigned']

,index,Postal Code,Borough,Neighborhood


In [25]:
df.shape

(103, 4)

# Part 2

In [9]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 10.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


Import geocoder lib to fetch latitude and longitude

In [26]:
import geocoder

geocoder wasn't working that's why using csv file given in the course

In [27]:
url = 'http://cocl.us/Geospatial_data'

In [17]:
def coords(postal_code):
     lat_lng_coords = None
     
     # loop until you get the coordinates
     while(lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
            lat_lng_coords = g.latlng

     return lat_lng_coords

Checking dtypes of our columns

In [ ]:
print(coords(df['Postal Code'][0]))

In [28]:
df_geo = pd.read_csv(url)
df_geo.head()

HTTPError: HTTP Error 308: Permanent Redirect

Shapes of both our Dataframes

In [207]:
df.shape

(103, 4)

In [208]:
df_geo.shape

(103, 3)

Joining both df's and doing some cleaning

In [209]:
df = df.join(df_geo.set_index('Postal Code'), on='Postal Code')
df

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,43.753259,-79.329656
1,3,M4A,North York,Victoria Village,43.725882,-79.315572
2,4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,165,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,168,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [210]:
df = df.reset_index()

In [211]:
df.drop(['index'], axis = 'columns', inplace = True)

In [212]:
df = df.set_index('level_0')

In [213]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
level_0,,,,,
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [214]:
df = df.rename(index = {'level_0' : 'index'})

In [215]:
df.index.name = 'index'

In [216]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
index,,,,,
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [217]:
df.shape

(103, 5)